# Проект по классификации текстов

***Цель:*** Обучить модель классифицировать комментарии на позитивные и негативные.

Значением метрики качества *F1* должно быть не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from nltk.corpus import stopwords 
from lightgbm import LGBMClassifier
from sklearn.pipeline import make_pipeline
from nltk.stem import WordNetLemmatizer 
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize 
from sklearn.model_selection import RandomizedSearchCV
import spacy
import warnings
from tqdm import notebook

In [2]:
warnings.filterwarnings("ignore")

Прочту данные, а также проверю пропуски и дубликаты.

In [3]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
data = pd.read_csv('/datasets/toxic_comments.csv', index_col = 0)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [6]:
data.isna().sum()

text     0
toxic    0
dtype: int64

In [7]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [8]:
data.duplicated().sum()

0

In [9]:
data['text'].duplicated().sum()

0

Напишу функцию для очистки и лемматизации текста:

In [10]:
nlp = spacy.load("en_core_web_sm")

def lemmatize(text):
    cleaned = re.sub(r'[^a-zA-Z ]', ' ', text)
    lowered = cleaned.lower()

    doc = nlp(lowered)
    lemmatized = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    return ' '.join(lemmatized)

In [11]:
data['lemm_text'] = data['text'].apply(lemmatize)

Данные обработаны, можно приступать к обучению.

## Обучение

Разобью данные на выборки, после чего создам два словаря гиперпараметров - для логистической регрессии и градиентного бустинга, также напишу функцию для кросс-валидации модели с помощью RandomizedSearchCV, в этой же функции воспользуюсь TfidfVectorizer.

In [12]:
X = data.drop(['toxic'], axis=1)
y = data['toxic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [13]:
lgbm_param_grid = {
    'lgbmclassifier__num_leaves': [31, 50],
    'lgbmclassifier__learning_rate': [0.1, 0.01, 0.005]
}

logreg_param_grid = {
    'logisticregression__C': [0.1, 1, 10],
    'logisticregression__penalty': ['l2'],
    'logisticregression__solver': ['lbfgs', 'liblinear'],
    'logisticregression__max_iter': [100, 200, 500]
}


In [14]:
def train_model(estimator, hyper_grid):
    
    pipeline = make_pipeline(TfidfVectorizer(), estimator)
    search = RandomizedSearchCV(pipeline, hyper_grid, cv=5, n_jobs=-1, scoring='f1')
    search.fit(X_train['lemm_text'], y_train)
    
    return search

***В ячейке ниже очень ресурсоемкая операция, которая заняла несколько часов***

In [15]:
logreg_model = train_model(LogisticRegression(random_state=42), logreg_param_grid)
logreg_model.best_params_

{'logisticregression__solver': 'lbfgs',
 'logisticregression__penalty': 'l2',
 'logisticregression__max_iter': 100,
 'logisticregression__C': 10}

In [16]:
logreg_model.best_score_

0.7758537044195486

***В ячейке ниже очень ресурсоемкая операция, которая заняла несколько часов***

In [17]:
lgbm_model = train_model(LGBMClassifier(random_state=42), lgbm_param_grid)
lgbm_model.best_params_

{'lgbmclassifier__num_leaves': 50, 'lgbmclassifier__learning_rate': 0.1}

In [18]:
lgbm_model.best_score_

0.7638098902175064

In [19]:
logreg_model.cv_results_['mean_fit_time']

array([82.98655715, 45.54704695, 11.76418047,  8.43668714, 24.16673045,
       24.18126183, 23.86407442,  8.36855731,  8.57626266, 46.49646544])

In [20]:
lgbm_model.cv_results_['mean_fit_time']

array([148.30674934, 192.13748765, 164.8950345 , 215.36172519,
       144.31836829, 228.26971722])

Лучшую метрику f1_score показала логистическая регрессия ~ 0.775. Градиентный бустинг показал результат чуть хуже и его обучение занимает значительно больше времени.

Проверю лучшую модель на тестовой выборке:

In [23]:
f1_score(y_test, logreg_model.predict(X_test['lemm_text']))

0.7749740753543035

Модель показала отличный f1_score равный 0.77.

## Выводы

В рамках проекта по классификации комментариев данные были лемматизированы с помощью WordNetLemmatizer, этот лемматизатор позволяет эффективно работать с текстами на английском языке. Была проведена кросс-валидация модели градиентного бустинга и логистической регрессии. На тестовой выборке лучшую F1‑меру ~ 0.77 показала логистическая регрессия. Это решение позволяет эффективно выделять токсичные комментарии для их последующей модерации, снижая нагрузку на модераторов.